In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

Set up interactive controls to use the interactive demo in this notebook. [Setup instructions here](https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6)

In [ ]:
import pandas as pd
import seaborn as sns

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from search import parse_demand, get_weight
from retrieval import Retrieval

### Input fields

* Demand data - coming from csv
* Weights data - coming from csv

In [ ]:
demand_data = pd.read_csv("../data/demand.csv", sep="\t")
demand_data

In [ ]:
weights_data = pd.read_csv("../data/weights.csv", sep="\t")
weights_data

### Interactive Demo

We will use the above input files to demo the search functionality.

In [ ]:
@interact
def show_search_results(demand_idx=range(7), weight_idx=range(3), sort_by_dept=False):
    obj = Retrieval()
    demand = parse_demand("../data/demand.csv", demand_idx)
    weight = get_weight("../data/weights.csv", weight_idx)
    res = obj.get_results(demand, weight, sort_by_dept).rename(
        columns={"years_of_experience": "experience"}
    )
    cm = sns.light_palette("green", as_cmap=True)
    return (
        res.style.background_gradient(cmap=cm, subset=["fitment"])
        .format(
            {
                "fitment": "{:.1%}",
                "technical": "{:.0f}",
                "functional": "{:.0f}",
                "process": "{:.0f}",
            }
        )
        .hide_index()
        .hide_columns("dept_similarity")
    )